# 03_欠損値を複数の参照dfを参照して穴埋めする関数を作成

## 目的

- tblのNanをmst1,mst2で補完埋めしたい。

## 条件

- tblのvalがNanのとき、
- cdがfで始まる時は、mst1のvalで補完埋めします
- cdがeで始まる時は、mst2のvalで補完埋めします
- それ以外のときはNanのままにします。

## 1. 入力：データフレームを作成

In [1]:
# はじめに
#
# この一連の欠損値の穴埋めをみてください

import numpy as np
import pandas as pd



tbl

,cd,val
0,F001,None
1,E001,None
2,X001,None
3,F001,
4,E001,
5,X001,
6,F001,100
7,E001,200
8,X001,300


### 出力：理想形

In [2]:
# # 期待する結果の更新用df状態です
# tbl = pd.DataFrame(
#     [
#     {"cd":"F001","val":1}, 
#     {"cd":"E001","val":4},
#     {"cd":"X001","val":None},
#     {"cd":"F001","val":1}, 
#     {"cd":"E001","val":4},
#     {"cd":"X001","val":None},
#     {"cd":"F001","val":100}, 
#     {"cd":"E001","val":200},
#     {"cd":"X001","val":300}
#     ],columns=["cd","val"]
# )
# tbl

In [3]:
# 'いけね、'(空文字)は欠損値ではないのか？この対処はこちらでやります。

# 後の確認等をやりやすくするために関数化しました。
import numpy as np
import pandas as pd

def create_tbl():
    tbl = pd.DataFrame(
        [
        {"cd":"F001","val":None}, 
        {"cd":"E001","val":None},
        {"cd":"X001","val":None},
        {"cd":"F001","val":''}, 
        {"cd":"E001","val":''},
        {"cd":"X001","val":''},
        {"cd":"F001","val":100}, 
        {"cd":"E001","val":200},
        {"cd":"X001","val":300}
        ],columns=["cd","val"]
    )
    tbl = tbl.replace({'': None})
    return tbl

tbl = create_tbl()
tbl

,cd,val
0,F001,NaN
1,E001,NaN
2,X001,NaN
3,F001,NaN
4,E001,NaN
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


## 2. 処理：データ型を変えずに穴埋め

In [4]:
# ルール１　tblのvalの欠損値の時に穴埋め処理をする。具体的なルールは以下の３つ（ルール２からルール４）。
# ルール２　tblのcdの値がFで始まる文字列の場合（例：F001）の場合は、fmstのval値で（例：F001なので1）欠損値の穴埋めをする
# ルール３　tblのcdの値がEで始まる文字列の場合（例：E001）の場合は、emstのval値で（例：E001なので4）欠損値の穴埋めをする
# ルール４　tblのcdの値の始まりがFでもEでもない場合（例：X001）は欠損値のままにしておく。

### データフレームの結合による処理

In [5]:
tbl = create_tbl()
tbl

,cd,val
0,F001,NaN
1,E001,NaN
2,X001,NaN
3,F001,NaN
4,E001,NaN
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


In [6]:
import numpy as np
import pandas as pd
import math


def is_not_nan(x):
    """
    func
    mapメソッドのために「NaNでなければTrueを返す」関数
    In
    x ; NaNでないことを判定したい値
    Out
    bool値
    """
    return not math.isnan(x)


def set_new_vel(tbl_merged, x_val):
    """
    func
    マージ後のdfにおいて、val列がNaNかつfmstまたはemstの値があるとき、
    dfのnew_val列にfmstまたはemstの値を代入する
    In
    tbl_merged : fmst, emstをマージしたtbl
    x_val : "f_val"または"e_val"を指定
    Out
    なし（tbl_mergedが直接修正される）
    """
    tbl_merged.loc[tbl_merged["val"].map(math.isnan) & tbl_merged[x_val].map(is_not_nan), "new_val"] = \
        tbl_merged.loc[tbl_merged["val"].map(math.isnan) & tbl_merged[x_val].map(is_not_nan), x_val]
    

# 03_table処理用の関数
# 参考：http://sinhrks.hatenablog.com/category/Python?page=1422712114
# 参考：https://note.nkmk.me/python-pandas-map-applymap-apply/
# 参考：http://sinhrks.hatenablog.com/entry/2015/07/11/223124

def fillnan_mst(tbl, fmst, emst):
    """
    func
    tblのvalがNanのとき、
    cdが'F'で始まる時は、fmstのvalで補完埋めします
    cdが'E'で始まる時は、emstのvalで補完埋めします
    それ以外のときはNanのままにします。
    In
    tbl : columns=["cd","val"]のpd.DataFrame（穴埋め更新対象）
    fmst : columns=["cd","val"]のpd.DataFrame（参照用）
    emst : columns=["cd","val"]のpd.DataFrame（参照用）
    Out
    fmst, emstを参照してNanを補完埋めしたtbl
    """
    # tblのval列と区別するためにfmst, emstのval列の列名を変更
    fmst_fval = fmst.rename(columns={'val': 'f_val'})
    emst_eval = emst.rename(columns={'val': 'e_val'})

    # tblにfmst, emstをマージ（結合）する
    tbl_merged = pd.merge(tbl, fmst_fval, on='cd', how='left')
    tbl_merged = pd.merge(tbl_merged, emst_eval, on='cd', how='left')
    
    # 各行から適切なval値を抽出し、new_val列に保存
    set_new_vel(tbl_merged, "f_val")
    set_new_vel(tbl_merged, "e_val")
    tbl_merged.loc[tbl_merged["val"].map(is_not_nan), "new_val"] = \
        tbl_merged.loc[tbl_merged["val"].map(is_not_nan), "val"]

    # 必要な列のみを抽出し、新しいdfを作成
    tbl = tbl_merged[["cd", "new_val"]]
    tbl = tbl.rename(columns={'new_val': 'val'})

    return tbl

## 3. 出力：各データのデータ型を確認

### ベースのdfで確認

In [7]:
tbl = create_tbl()

%time tbl = fillnan_mst(tbl, fmst, emst)

tbl

Wall time: 35 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


### cdの値がF001～F003, E001～E003まである場合を確認

In [8]:
def create_table_large():
    tbl_large = pd.DataFrame(
        [
        {"cd":"F001","val":None}, 
        {"cd":"E001","val":None},
        {"cd":"X001","val":None},
        {"cd":"F001","val":''}, 
        {"cd":"E001","val":''},
        {"cd":"X001","val":''},
        {"cd":"F001","val":100}, 
        {"cd":"E001","val":200},
        {"cd":"X001","val":300},
        {"cd":"F002","val":None}, 
        {"cd":"E002","val":None},
        {"cd":"X002","val":None},
        {"cd":"F002","val":''}, 
        {"cd":"E002","val":''},
        {"cd":"X002","val":''},
        {"cd":"F002","val":100}, 
        {"cd":"E002","val":200},
        {"cd":"X002","val":300},
        {"cd":"F003","val":None}, 
        {"cd":"E003","val":None},
        {"cd":"X003","val":None},
        {"cd":"F003","val":''}, 
        {"cd":"E003","val":''},
        {"cd":"X003","val":''},
        {"cd":"F003","val":100}, 
        {"cd":"E003","val":200},
        {"cd":"X003","val":300},
        ],columns=["cd","val"]
    )
    tbl_large = tbl_large.replace({'': None})
    
    return tbl_large

In [9]:
tbl_large = create_table_large()

%time tbl_large = fillnan_mst(tbl_large, fmst, emst)

tbl_large

Wall time: 29 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,2.0


---

### 参考：データフレームの結合による処理（applyメソッドを使用した、時間がかかる処理）

参考：http://sinhrks.hatenablog.com/entry/2015/07/11/223124

In [10]:
import numpy as np
import pandas as pd
import math

# mergeしたdfの各行から適切な値を抽出する関数（pd.Seriesに適用）
def get_new_series_val(series):
    """
    func
    mergeしたdfの各行から適切な値を抽出する
    In
    series : columns=["cd","val", "f_val", "e_val"]のpd.Series（tbl, fmst, emstをmergeしたdfの各行）
    out
    tblのvalに代入される適切な値
    """
    if math.isnan(series.val) and not math.isnan(series.f_val):
        new_val = series.f_val
    elif math.isnan(series.val) and not math.isnan(series.e_val):
        new_val = series.e_val
    else:
        new_val = series.val

    return new_val

In [11]:
# 03_table処理用の関数
# 参考：http://sinhrks.hatenablog.com/category/Python?page=1422712114
# 参考：https://note.nkmk.me/python-pandas-map-applymap-apply/

def fillnan_mst_merge_apply(tbl, fmst, emst):
    """
    func
    tblのvalがNanのとき、
    cdが'F'で始まる時は、fmstのvalで補完埋めします
    cdが'E'で始まる時は、emstのvalで補完埋めします
    それ以外のときはNanのままにします。
    In
    tbl : columns=["cd","val"]のpd.DataFrame（穴埋め更新対象）
    fmst : columns=["cd","val"]のpd.DataFrame（参照用）
    emst : columns=["cd","val"]のpd.DataFrame（参照用）
    out
    fmst, emstを参照してNanを補完埋めしたtbl
    """
    # tblのval列と区別するためにfmst, emstのval列の列名を変更
    fmst_fval = fmst.rename(columns={'val': 'f_val'})
    emst_eval = emst.rename(columns={'val': 'e_val'})

    # tblにfmst, emstをマージ（結合）する
    tbl_merged = pd.merge(tbl, fmst_fval, on='cd', how='left')
    tbl_merged = pd.merge(tbl_merged, emst_eval, on='cd', how='left')
    
    # 各行から適切なval値を抽出し、new_val列に保存
    tbl_merged["new_val"] = tbl_merged.apply(get_new_series_val, axis=1)

    # 必要な列のみを抽出し、新しいdfを作成
    tbl = tbl_merged[["cd", "new_val"]]
    tbl = tbl.rename(columns={'new_val': 'val'})
    
    return tbl

### ベースのdfで確認

In [12]:
tbl = create_tbl()

%time tbl = fillnan_mst_merge_apply(tbl, fmst, emst)

tbl

Wall time: 19 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


### cdの値がF001～F003, E001～E003まである場合を確認

In [13]:
tbl_large = create_table_large()

%time tbl_large = fillnan_mst_merge_apply(tbl_large, fmst, emst)

tbl_large

Wall time: 21 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,2.0


### スピードテスト

In [14]:
# 約27,000行のdfを作成
tbl_large_large = create_table_large()
tmp = create_table_large()
for i in range(1000):
    tbl_large_large = tbl_large_large.append(tmp)

In [15]:
tbl_large_large.head(100)

,cd,val
0,F001,NaN
1,E001,NaN
2,X001,NaN
3,F001,NaN
4,E001,NaN
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,NaN


In [16]:
tbl_large_large_copy = tbl_large_large.copy()

%time tbl_large_large_copy = fillnan_mst(tbl_large_large_copy, fmst, emst)

tbl_large_large_copy.head(100)

Wall time: 161 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,2.0


In [17]:
tbl_large_large_copy = tbl_large_large.copy()

%time tbl_large_large_copy = fillnan_mst_merge_apply(tbl_large_large_copy, fmst, emst)

tbl_large_large_copy.head(100)

Wall time: 2.38 s


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,2.0


---

## 参考：for文による処理

マージ（結合）による処理ができたのでfor文による処理は不要

参考程度に。

In [18]:
import math

def fillnan_mst_for(tbl, fmst, emst):
    """
    func
    tblのvalがNanのとき、
    cdが'F'で始まる時は、fmstのvalで補完埋めします
    cdが'E'で始まる時は、emstのvalで補完埋めします
    それ以外のときはNanのままにします。
    In
    tbl : columns=["cd","val"]のpd.DataFrame（穴埋め更新対象）
    fmst : columns=["cd","val"]のpd.DataFrame（参照用）
    emst : columns=["cd","val"]のpd.DataFrame（参照用）
    out
    fmst, emstを参照してNanを補完埋めしたtbl
    """
    for i in range (len(tbl)):
        if math.isnan(tbl.loc[i, "val"]):
            if tbl.loc[i, "cd"].startswith("F"):
                tbl.loc[i, "val"] = fmst[fmst["cd"] == tbl.loc[i, "cd"]]["val"].values[0]
            elif tbl.loc[i, "cd"].startswith("E"):
                tbl.loc[i, "val"] = emst[emst["cd"] == tbl.loc[i, "cd"]]["val"].values[0]
        
    return tbl

### ベースのdfで確認

In [19]:
tbl = create_tbl()
tbl

,cd,val
0,F001,NaN
1,E001,NaN
2,X001,NaN
3,F001,NaN
4,E001,NaN
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


In [20]:
tbl = create_tbl()

%time tbl = fillnan_mst_for(tbl, fmst, emst)

tbl

Wall time: 21 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


### cdの値がF001～F003, E001～E003まである場合を確認

In [21]:
tbl_large = create_table_large()
tbl_large

,cd,val
0,F001,NaN
1,E001,NaN
2,X001,NaN
3,F001,NaN
4,E001,NaN
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,NaN


In [22]:
tbl_large = create_table_large()

%time tbl_large = fillnan_mst_for(tbl_large, fmst, emst)

tbl_large

Wall time: 29 ms


,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
9,F002,2.0


---

## ベクトル操作による処理

※断念

マージ（結合）による処理ができたのでベクトル操作による処理は不要

---

In [ ]:
# 実験１：tblのcdがFで始まる文字列
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":None}, 
    {"cd":"E001","val":None},
    {"cd":"X001","val":None},
    {"cd":"F001","val":''}, 
    {"cd":"E001","val":''},
    {"cd":"X001","val":''},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
tbl[tbl['cd'].str.startswith("F")]
# 結果はOKなようです


In [ ]:
# 実験２：tblのvalが欠損値
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":None}, 
    {"cd":"E001","val":None},
    {"cd":"X001","val":None},
    {"cd":"F001","val":''}, 
    {"cd":"E001","val":''},
    {"cd":"X001","val":''},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
tbl['val'].isnull()
#　結果はOKなようです

In [ ]:
# 実験３
# tblでcdがFで始まる文字列のValにスカラー値を代入する
# 参考情報　https://note.nkmk.me/python-pandas-where-mask/
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":None}, 
    {"cd":"E001","val":None},
    {"cd":"X001","val":None},
    {"cd":"F001","val":''}, 
    {"cd":"E001","val":''},
    {"cd":"X001","val":''},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
tbl.loc[tbl['cd'].str.startswith("F"),'val' ] = -100
tbl

In [ ]:
# 実験４
# blでcdがFで始まる文字列で、valがNanにスカラー値を代入する
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":None}, 
    {"cd":"E001","val":None},
    {"cd":"X001","val":None},
    {"cd":"F001","val":''}, 
    {"cd":"E001","val":''},
    {"cd":"X001","val":''},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
#tbl.loc[ tbl['cd'].str.startswith("F") & True ,'val' ] = -100
#
# このTrueにtbl.isnull()
#
tbl.loc[ tbl['cd'].str.startswith("F") & tbl['val'].isnull() ,'val' ] = -100
tbl
#
# ここまではできました。
#  cdの文字列の初めの文字は判断できます
#  valが欠損値なのも判断できます
# その条件に剃ってスカラー値を代入することもできます

In [ ]:
# 実験５　参照用dfから値を取得する
#

# 参照df その１
fmst = pd.DataFrame(
    [
    {"cd":"F001","val":1}, 
    {"cd":"F002","val":2},
    {"cd":"F003","val":3}
    ],columns=["cd","val"]
)
key = "F002"
fmst['cd'] == key 

# tblの埋めに使う値はcdをキーに参照用dfから持ってくる必要があります。